# SRL Benchmarking (Colab)
Run vLLM-based benchmarks for SRL/SFT models, saving results for later plotting.

## 0. Setup
- Runtime: GPU (A100 recommended)
- HF token: set `HF_TOKEN` env var if needed for private models/datasets.
- Repo: cloned into `/content/SRL-reasoning`.

In [ ]:
# --- Colab setup (run this first in a fresh runtime) ---
REPO_URL = "https://github.com/iroblesrazzaq/SRL-reasoning.git"  # update if needed
BRANCH = "main"                               # update if needed
WORKDIR = "/content/SRL-reasoning"

# Install before importing anything
%pip install -U pip
import os

if not os.path.exists(WORKDIR):
    !git clone --branch $BRANCH $REPO_URL $WORKDIR
%cd $WORKDIR
%pip install -e .

from pathlib import Path
import time
from benchmarks import load_benchmark_data, MathEvaluator, BenchmarkResult, data_loader
import os as _os
import shutil


print("Ready. Repo at", WORKDIR)

## 1. (Optional) Mount Drive for saving results/checkpoints

In [ ]:

MOUNT_DRIVE = True  # set True to mount
if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_RESULTS_DIR = '/content/drive/MyDrive/srl_bench_results'
else:
    DRIVE_RESULTS_DIR = None


## 2. Configs - for fine-tuned model, must set path to local path (mount from drive?)

In [ ]:

MODEL_PATH = "Qwen/Qwen3-4B-Instruct-2507"  # e.g., "yourname/srl-lora" or "/content/ckpts/best"
MODEL_NAME = "Qwen3-4B-Instruct-2507"  # optional display name; defaults to basename of MODEL_PATH

# TODO: load trained SFT/SRL models, name them as f"{MODEL_NAME}-sft" or f"{MODEL_NAME}-srl"

MODEL_TYPE = "base"            # choices: srl (has <think>), base (no <think>) TODO: select srl if using SRL model
GPU_MEMORY_UTILIZATION = 0.8  
RESULTS_DIR = "benchmarks/results"

SEED = 42


## 3. Instantiate evaluator (vLLM allocates memory)

In [ ]:
_gpu_util = str(GPU_MEMORY_UTILIZATION)
_os.environ.setdefault("VLLM_GPU_MEMORY_UTILIZATION", _gpu_util)

evaluator = MathEvaluator(
    MODEL_PATH,
    model_type=MODEL_TYPE,
    gpu_memory_utilization=GPU_MEMORY_UTILIZATION
)

### 4. benchmarking loop

In [ ]:
BENCHMARKS = ["aime24", "aime25", 'amc23']          
MODES = ["greedy", "avg32"]   
MODES = ["greedy"] # for testing

for benchmark in BENCHMARKS:
    for mode in MODES:
        eval_start = time.time()
        data = load_benchmark_data(benchmark)
        #print(f"Running mode='{mode}' for benchmark='{benchmark}' ...")
        score = evaluator.evaluate(data, mode=mode)
        eval_end = time.time() - eval_start
        #print(f"Score: {score:.4f} | elapsed: {eval_end/60:.1f} min")
        benchmark_type = "Avg32" if mode == "avg32" else "Greedy"

        model_display = MODEL_NAME or Path(MODEL_PATH).name

        result = BenchmarkResult(
            benchmark=benchmark,
            benchmark_type=benchmark_type,
            score=score,
            model_name=model_display,
            model_path=MODEL_PATH,
            num_questions=len(data),
            eval_time_seconds=eval_end,
            seed=SEED,
        )

        path = result.save(RESULTS_DIR)
        print("Saved result to", path)

        # save to drive every benchmark in case of crash
        if DRIVE_RESULTS_DIR:
            dest = Path(DRIVE_RESULTS_DIR)
            dest.mkdir(parents=True, exist_ok=True)
            for f in Path(RESULTS_DIR).glob("*.json"):
                shutil.copy2(f, dest / f.name)
            print("Copied results to", dest)
        else:
            print("Drive not mounted; skipping copy.")




### 4. benchmark over all datasets, modes

## 5. Inspect saved results

In [ ]:

from benchmarks import load_all_results, summarize_results

results = list(load_all_results(RESULTS_DIR))
print(f"Loaded {len(results)} result file(s)")
for r in results:
    print(f"- {r.benchmark} | {r.benchmark_type} | {r.model_name} | score={r.score:.4f} | run_id={r.run_id}")

best = summarize_results(results)
print("Best scores (benchmark, model_name) -> score:")
for (bench, model), sc in best.items():
    print(f"{bench} / {model}: {sc:.4f}")
